# Domain coverage with coordinate-free sensors

Sensor coverage problem solved by means of the Rips complex and its relative homology in a coordinate-free scenario.

This is an implementation of the ideas in the paper **_Coordinate-free coverage in sensor networks with controlled boundaries via homology_** by **V. De Silva** and **R. Ghrist**.

## Problem statement

Given a compact connected domain $D \subset \mathbb R^2$ we wish to know whether a set of sensors (nodes) $X \subset D$ with a lack of localization capabilities cover (contain) $D$. Four assumptions are made:

1. Nodes $X$ are uniquely identifiable. Each node can detect the identity of any node with **broadcast radius** $r_b$, but they cannot determine the direction nor distance to that neighboring node.
2. Nodes have have a covering radius which covers parts of the domain $r_c \ge \frac{r_b}{\sqrt 3}$.
3. Boundary of $D$ denoted by $\partial D$ is connected and piecewise-linear with vertices marked **fence nodes** $X_f$ (they form a cycle).
4. Each fence node $v \in X_f$ is within $r_b$ distance of its two neighbors.

The question then is to determine whether the sensors cover the domain, ie.

$$
\bigcup_{x \in X} D^2(x, r_c) \stackrel{?}{\supseteq} D
$$

Where $D^2(x, R)$ denotes a disk of radius $r_c$ centered at $x$; the coverage of a sensor at $x$.

## Solution

We construct the Rips complex $\mathcal R$ from the communication graph of nodes $X$ and the radius $r_b$; two nodes are connected if they are within the distance $r_b$. We also construct the Rips complex of the fence nodes $X_f$, which is simply a 1-cycle denoted here by $\mathcal F$. Of course, $\mathcal F \subset \mathcal R$. Then we apply the theorem derived in the paper:

> The union of the radius $r_c$ disks contains $D$ if there is a nontrivial element of the relative homology $H_2(\mathcal R, \mathcal F)$ whose boundary is nonvanishing.

### Remarks

Authors note that the bound $r_c \ge \frac{r_b}{\sqrt 3}$ (consider the case of an equilateral triangle) together with the verification of holes in $\mathcal R$ (ie $H_1(\mathcal R) = \mathbf 0$) is enough to conclude that the coverage of $D$ is obtained. But the criterion considering the relative homology of complexes of nodes and the fence is a stronger one. In both cases because $r_c$ does not necessarily equal to $r_b$ the criterion is only a sufficient condition, not a necessary one.

## Load sensors

We first load the JSON file describing the sensors network created with the sensors network creator. Feel free to change the loaded file to verify the computation for other cases.

In [112]:
from dataclasses_json import dataclass_json
from dataclasses import dataclass

@dataclass_json
@dataclass(frozen=True)
class Point:
    x: int
    y: int
    fence: bool

@dataclass_json
@dataclass(frozen=True)
class SensorsData:
    coverage_radius: int
    broadcast_radius: int
    sensors: list[Point]

with open('../data/not_covered_sensors.json', 'r') as f:
    data: SensorsData = SensorsData.from_json(f.read())



## Draw the sensors

We visualize the sensors, their broadcast, and their coverage.

In [113]:
from ipycanvas import Canvas, hold_canvas

canvas = Canvas(width=700, height=500)


for p in data.sensors:
    canvas.fill_style = "rgba(130, 245, 216, 0.4)"
    canvas.fill_circle(p.x, p.y, data.coverage_radius)
for p in data.sensors:
    canvas.fill_style = "rgba(103, 157, 245, 0.4)"
    canvas.fill_circle(p.x, p.y, data.broadcast_radius)
for p in data.sensors:
    canvas.fill_style = "black"
    canvas.fill_circle(p.x, p.y, 3)

canvas

Canvas()

## Construct the Rips complex

The theorem can be interpreted the following way: $\dim H_2(\mathcal R, \mathcal F) > 0$ and there exists a generator $[\alpha] \in H_2(\mathcal R, \mathcal F)$ such that $\partial\alpha \ne 0$. `gudhi` does not support calculation of relative homology but using the excision theorem we can calculate this homology. The theorem says that the relative homology is isomorphic to the reduced homology of the quotient space: $\tilde H_*(X / A) \simeq H_*(X, A)$. Since $\tilde H_k \simeq H_k$ for $k > 0$ and we wish to consider $k=2$, we can use this theorem by construction of $\mathcal R / \mathcal F$.

Thanks to homotopy invariance it is enough if we construct a complex whose homotopy type is that of the quotient space $\mathcal R / \mathcal F$. For example, one where we identify all fence nodes. To do so, to the $\mathcal R$ complex we add a disjoint vertex which we then connect to every simplex in $\mathcal F$.

Note: we directly use the coordinates here, but this is merely for simulation purposes. The Rips complex could be constructed without these coordinates, all we need to know is whether some sensor is within the broadcast radius.

We start by separating the fence and inside nodes. As a sanity check of the fence complex we verify whether it is a 1-cycle.

In [114]:
import gudhi as gd

inside = [(o.x, o.y) for o in data.sensors if not o.fence]
fence = [(o.x, o.y) for o in data.sensors if o.fence]

def remove_persistence(cmplx: gd.SimplexTree):
    for (simplex, _) in cmplx.get_filtration():
        cmplx.assign_filtration(simplex, 0)


F = gd.RipsComplex(points=fence, max_edge_length=data.broadcast_radius)
F = F.create_simplex_tree(len(fence))
# we don't care about persistance, this sets all filtration values to zero
F.reset_filtration(0)
F.compute_persistence(persistence_dim_max=True)

# 1-cycle
assert F.dimension() == 1 and F.betti_numbers() == [1, 1]


Once the fence is OK we proceed to construct $\mathcal R / \mathcal F$ (or rather something with equivalent properties) by constructing the cone over simplices of $\mathcal F$ as described before.

In [117]:
R = gd.RipsComplex(points=[*inside, *fence], max_edge_length=data.broadcast_radius)
R = R.create_simplex_tree(len(inside) + len(fence))
R.reset_filtration(0)

RF = R.copy()

# find all fence simplices and connect them to a new vertex
for (simplex, _) in R.get_simplices():
    if len(simplex) != 2 or not all([x >= len(inside) for x in simplex]):
        continue
    RF.insert([*simplex, len(inside) + len(fence)])

RF.compute_persistence(persistence_dim_max=True)

Now can can verify the condition $\dim H_2(\mathcal R, \mathcal F) = \dim H_2(\mathcal R / \mathcal F) > 0$

In [119]:
print(f'Is dim(H_2(R, F)) > 0?', RF.betti_numbers()[2] > 0)

Is dim(H_2(R, F)) > 0? False
